In [1]:
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import supervised_convnet
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import sys
from sklearn.model_selection import train_test_split



In [15]:
ones = np.ones((10000, 9))
not_ones = np.random.choice([1, -1], (10000, 9))
data = np.vstack((ones, not_ones))
label = np.hstack((np.zeros(10000), np.ones(10000)))
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.33, random_state=42)

In [20]:
class OnesDataset(Dataset):
    def __init__(self, data, label):
        self.X = data
        self.y = label
        
        
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
    def __len__(self):
        return len(self.X)
   
OnesDataset = OnesDataset(X_train, y_train)
OnesDataset.X


array([[ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       ...,
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1., -1., -1., ..., -1., -1.,  1.]])

In [32]:
class SupervisedConvNet(nn.Module):
    def __init__(self, filter_size, square_size):
        super(SupervisedConvNet, self).__init__()
        self.filter_size = filter_size
        self.square_size = square_size
        self.conv2d = nn.Conv2d(1, 1, filter_size, padding=0, stride = filter_size)  
        self.linear = nn.Linear(filter_size ** 2, 1)
        

        
    def forward(self, x):
        # add hidden layers with relu activation function
        linear = self.linear(x)
        output = F.sigmoid(linear)
        # for row in x:
        #     print("row", row)
        #     for el in row[0]:
        #         print("el", el)
        # x = torch.tanh(self.decoder(x))
        return output

model = SupervisedConvNet(filter_size = 3, square_size = 9)

In [33]:
# specify loss function
criterion = nn.BCELoss()

# specify loss function
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# Create training and test dataloaders
num_workers = 0
# how many samples per batch to load
batch_size = 20
# number of epochs to train the model
n_epochs = 50

# prepare data loaders
train_loader = torch.utils.data.DataLoader(OnesDataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)

for epoch in range(1, n_epochs+1):
    # monitor training loss
    train_loss = 0.0

    ###################
    # train the model #
    ###################
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.unsqueeze(1).type('torch.FloatTensor')
        target = target.type('torch.FloatTensor')
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        # update running training loss
        train_loss += loss.item() * batch_size
    
    # print avg training statistics 
    train_loss = train_loss/len(train_loader)
    if epoch % 1 == 0:
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(
            epoch, 
            train_loss
            ))
#         print("data", data[:10])
#         print("output", (output)[:10])
#         print("target", (target)[:10])
        for name, param in model.named_parameters():
            if param.requires_grad:
                print (name, param.grad)

Epoch: 1 	Training Loss: 3.459421
conv2d.weight None
conv2d.bias None
linear.weight tensor([[0.0115, 0.0181, 0.0277, 0.0294, 0.0125, 0.0094, 0.0190, 0.0150, 0.0399]])
linear.bias tensor([0.0062])
Epoch: 2 	Training Loss: 1.638845
conv2d.weight None
conv2d.bias None
linear.weight tensor([[ 0.0042, -0.0071, -0.0355, -0.0080,  0.0451, -0.0440, -0.0058, -0.0455,
         -0.0460]])
linear.bias tensor([-0.0477])
Epoch: 3 	Training Loss: 1.285370
conv2d.weight None
conv2d.bias None
linear.weight tensor([[ 0.0572, -0.0184, -0.0190, -0.0190, -0.0191, -0.0191, -0.0191, -0.0180,
         -0.0184]])
linear.bias tensor([-0.0202])
Epoch: 4 	Training Loss: 1.111498
conv2d.weight None
conv2d.bias None
linear.weight tensor([[ 0.0232, -0.0036, -0.0004,  0.0036, -0.0057, -0.0055, -0.0040,  0.0003,
          0.0321]])
linear.bias tensor([-0.0057])
Epoch: 5 	Training Loss: 0.996917
conv2d.weight None
conv2d.bias None
linear.weight tensor([[0.0116, 0.0034, 0.0110, 0.0119, 0.0038, 0.0077, 0.0097, 0.0068, 0.

Epoch: 42 	Training Loss: 0.420199
conv2d.weight None
conv2d.bias None
linear.weight tensor([[0.0070, 0.0070, 0.0071, 0.0070, 0.0070, 0.0070, 0.0070, 0.0071, 0.0071]])
linear.bias tensor([0.0070])
Epoch: 43 	Training Loss: 0.415837
conv2d.weight None
conv2d.bias None
linear.weight tensor([[0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0035, 0.0077, 0.0035, 0.0077]])
linear.bias tensor([0.0035])
Epoch: 44 	Training Loss: 0.412589
conv2d.weight None
conv2d.bias None
linear.weight tensor([[0.0057, 0.0076, 0.0057, 0.0057, 0.0057, 0.0057, 0.0057, 0.0057, 0.0076]])
linear.bias tensor([0.0057])
Epoch: 45 	Training Loss: 0.408654
conv2d.weight None
conv2d.bias None
linear.weight tensor([[0.0078, 0.0055, 0.0055, 0.0055, 0.0055, 0.0055, 0.0055, 0.0078, 0.0055]])
linear.bias tensor([0.0055])
Epoch: 46 	Training Loss: 0.405020
conv2d.weight None
conv2d.bias None
linear.weight tensor([[0.0047, 0.0047, 0.0047, 0.0077, 0.0047, 0.0047, 0.0047, 0.0077, 0.0047]])
linear.bias tensor([0.0047])
Epoch: 47 	Trai

In [34]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.data)

conv2d.weight tensor([[[[-0.2395, -0.2458,  0.0759],
          [ 0.2451, -0.0515,  0.0264],
          [-0.0617, -0.1918,  0.1944]]]])
conv2d.bias tensor([0.0959])
linear.weight tensor([[-2.3003, -2.3589, -2.1069, -2.1754, -2.1448, -2.1971, -1.9458, -1.8449,
         -1.9532]])
linear.bias tensor([14.5680])
